In [2]:
import pandas as pd
import numpy as np
#from sklearn import linear_model
#from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import pickle
import gc
import psutil
import os
import time

In [3]:
indicator = "income"

In [4]:

# Step 1: Load the saved model
#f'../4-Train/{folder_indicator}/folds/{FOLD}.csv'
#Combine SL+SM/FOLD_SV_SM_SAT_AGG_RIDGE_RANDOM_14Aug23_2027_income/1/finalized_model.sav
#model_filename = f'../Combine SL+SM/FOLD_SV_SM_SAT_AGG_RIDGE_RANDOM_21Aug23_0113_income/1/finalized_model.sav'
#model_filename = f'../Combine SL+SM/FOLD_SV_SM_SAT_GROUPED_RIDGE_RANDOM_22Aug23_0104_income/1/finalized_model.sav'
model_filename = f'../Combine SL+SM/FOLD_SV_SM_SAT_AGG_500_RIDGE_RANDOM_29Aug23_1207_income/4/finalized_model.sav'
loaded_model = pickle.load(open(model_filename, 'rb'))


In [5]:
data =  pd.read_csv(f'../Combine SL+SM/FOLD_SV_SM_SAT_AGG_500_RIDGE_RANDOM_29Aug23_1207_income/4//data.csv')
data

,setor,image_name,pano_id,income,literacy,population,longevity
0,350270505000002,-24.515912057096592_-48.84748623854377_-24.421...,uE351968Ajhb8Hc5LNpQRA,0.671455,0.971920,452.0,0.850000
1,350270505000006,-24.523710645212436_-48.855999206534015_-24.42...,g25bUe5LZK9l1UeD1VgJNA,0.675544,0.808081,29.0,1.000000
2,350270505000024,-24.492516292749055_-48.84748623854377_-24.421...,qveA2BgacasXZOeXVFKvWA,0.489885,0.951034,483.0,0.850000
3,350270505000029,-24.461321940285675_-48.88153811050475_-24.421...,ErVcA0DfJZdVYB2cFvRAwg,0.461539,0.937810,357.0,0.783333
4,350270505000030,-24.453523352169828_-48.84748623854377_-24.421...,exJPaYypi5h_uPMLANCwMw,0.421362,0.897778,285.0,0.866667
...,...,...,...,...,...,...,...
105,412220605000047,-25.188312794713493_-49.16514186771816_-25.076...,eryzulyNqZW75Dig0jErtA,0.707800,0.983198,446.0,0.866667
106,412220605000049,-25.188312794713493_-49.16514186771816_-25.076...,FJUpDBbpxVsRv_cLxfN_sA,0.523324,0.920875,888.0,0.800000
107,412220605000051,-25.188312794713493_-49.16514186771816_-25.076...,qhWQCjXzlPJkwnJ5ZAkVzQ,0.537953,0.945469,614.0,0.783333
108,412788205000002,-24.973825915306104_-49.08338935207254_-24.952...,gW4-3m6Hw-0iPkjkK-VT-A,0.431575,0.965920,1170.0,0.800000


In [9]:
data['fold'] = 4

In [6]:

# Step 2: Load the DataFrame
df_filename = "df_sv_sat_sm_agg_vggs.csv" #SV+SAT+SM-AGG 28092 rows × 57352 columns
#df_filename = "df_sv_sm_sat_vgg.csv"       #SV+SAT+SM
df = pd.read_csv(df_filename)
df

,pano_id,setor,income,literacy,longevity,population,TIPO,v0_0,v0_90,v0_180,...,v4086,v4087,v4088,v4089,v4090,v4091,v4092,v4093,v4094,v4095
0,---e3e3zE_V5NU9hNKppsQ,352460005000005,0.660533,0.983471,0.850000,742.0,URBANO,1.572870,0.618193,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.108489,0.000000,0.000000,0.0,0.0
1,--3ujCBxxChD2G0H4nS93g,354260205000067,0.735173,0.994571,0.816667,777.0,URBANO,0.930203,1.123909,1.535309,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.126926,0.000000,0.190567,0.0,0.0
2,--9ZtMQo8qiKs675YC1xIg,412220605000008,0.499030,0.960421,0.750000,595.0,URBANO,0.385374,0.369581,0.699349,...,0.000000,0.000000,1.441954,0.051442,1.331690,1.070455,0.000000,0.000000,0.0,0.0
3,--AnK1A5iUOxkuU84IdsFA,352042605000008,0.525143,0.891971,0.833333,182.0,URBANO,0.000000,0.072973,0.172268,...,0.173760,0.000000,0.000000,0.000000,0.000000,0.000000,1.506161,1.873004,0.0,0.0
4,--CFI-vKfNNTAKSpkxNX2A,352460005000002,0.679411,0.940114,0.916667,784.0,URBANO,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.743313,0.000000,0.000000,0.000000,0.000000,1.854009,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28087,zym-GL-KBKRqLQDdTm5csg,412220605000050,0.513897,0.897385,0.800000,715.0,URBANO,0.000000,0.000000,0.957475,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.349837,1.875474,0.000000,0.0,0.0
28088,zz2USdjR81hiUpG4IRTPwQ,410310705000015,0.528199,0.956389,0.800000,458.0,RURAL,1.007803,0.068867,0.108461,...,1.438920,0.000000,0.000000,0.000000,0.000000,1.224538,0.000000,2.036786,0.0,0.0
28089,zz4P4Hi_eKM9OtoAL0ByNw,352610005000025,0.499226,0.902690,0.783333,427.0,RURAL,0.000000,0.000000,0.848774,...,0.258808,0.362669,0.000000,0.000000,0.000000,0.196430,0.000000,0.000000,0.0,0.0
28090,zzZ7kabHUev787MQtij-Tg,352030105000050,0.696957,0.981552,0.816667,724.0,URBANO,0.357998,1.288575,0.000000,...,0.000000,0.000000,1.532890,0.000000,0.267502,0.000000,0.000000,0.000000,0.0,0.0


In [10]:
df1 = df.merge(data[['setor', 'pano_id', 'image_name', 'fold']], on=['setor','pano_id','image_name'], how='left')
df1

,pano_id,setor,income,literacy,longevity,population,TIPO,v0_0,v0_90,v0_180,...,v4087,v4088,v4089,v4090,v4091,v4092,v4093,v4094,v4095,fold
0,---e3e3zE_V5NU9hNKppsQ,352460005000005,0.660533,0.983471,0.850000,742.0,URBANO,1.572870,0.618193,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.108489,0.000000,0.000000,0.0,0.0,NaN
1,--3ujCBxxChD2G0H4nS93g,354260205000067,0.735173,0.994571,0.816667,777.0,URBANO,0.930203,1.123909,1.535309,...,0.000000,0.000000,0.000000,0.000000,1.126926,0.000000,0.190567,0.0,0.0,NaN
2,--9ZtMQo8qiKs675YC1xIg,412220605000008,0.499030,0.960421,0.750000,595.0,URBANO,0.385374,0.369581,0.699349,...,0.000000,1.441954,0.051442,1.331690,1.070455,0.000000,0.000000,0.0,0.0,NaN
3,--AnK1A5iUOxkuU84IdsFA,352042605000008,0.525143,0.891971,0.833333,182.0,URBANO,0.000000,0.072973,0.172268,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.506161,1.873004,0.0,0.0,NaN
4,--CFI-vKfNNTAKSpkxNX2A,352460005000002,0.679411,0.940114,0.916667,784.0,URBANO,0.000000,0.000000,0.000000,...,0.000000,2.743313,0.000000,0.000000,0.000000,0.000000,1.854009,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28087,zym-GL-KBKRqLQDdTm5csg,412220605000050,0.513897,0.897385,0.800000,715.0,URBANO,0.000000,0.000000,0.957475,...,0.000000,0.000000,0.000000,0.000000,0.349837,1.875474,0.000000,0.0,0.0,NaN
28088,zz2USdjR81hiUpG4IRTPwQ,410310705000015,0.528199,0.956389,0.800000,458.0,RURAL,1.007803,0.068867,0.108461,...,0.000000,0.000000,0.000000,0.000000,1.224538,0.000000,2.036786,0.0,0.0,NaN
28089,zz4P4Hi_eKM9OtoAL0ByNw,352610005000025,0.499226,0.902690,0.783333,427.0,RURAL,0.000000,0.000000,0.848774,...,0.362669,0.000000,0.000000,0.000000,0.196430,0.000000,0.000000,0.0,0.0,NaN
28090,zzZ7kabHUev787MQtij-Tg,352030105000050,0.696957,0.981552,0.816667,724.0,URBANO,0.357998,1.288575,0.000000,...,0.000000,1.532890,0.000000,0.267502,0.000000,0.000000,0.000000,0.0,0.0,NaN


In [37]:
df2 = df1[df1['fold'].isna()]

In [38]:
df2

,pano_id,setor,income,literacy,longevity,population,TIPO,v0_0,v0_90,v0_180,...,v4087,v4088,v4089,v4090,v4091,v4092,v4093,v4094,v4095,fold
0,---e3e3zE_V5NU9hNKppsQ,352460005000005,0.660533,0.983471,0.850000,742.0,URBANO,1.572870,0.618193,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.108489,0.000000,0.000000,0.0,0.0,NaN
1,--3ujCBxxChD2G0H4nS93g,354260205000067,0.735173,0.994571,0.816667,777.0,URBANO,0.930203,1.123909,1.535309,...,0.000000,0.000000,0.000000,0.000000,1.126926,0.000000,0.190567,0.0,0.0,NaN
2,--9ZtMQo8qiKs675YC1xIg,412220605000008,0.499030,0.960421,0.750000,595.0,URBANO,0.385374,0.369581,0.699349,...,0.000000,1.441954,0.051442,1.331690,1.070455,0.000000,0.000000,0.0,0.0,NaN
3,--AnK1A5iUOxkuU84IdsFA,352042605000008,0.525143,0.891971,0.833333,182.0,URBANO,0.000000,0.072973,0.172268,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.506161,1.873004,0.0,0.0,NaN
4,--CFI-vKfNNTAKSpkxNX2A,352460005000002,0.679411,0.940114,0.916667,784.0,URBANO,0.000000,0.000000,0.000000,...,0.000000,2.743313,0.000000,0.000000,0.000000,0.000000,1.854009,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28087,zym-GL-KBKRqLQDdTm5csg,412220605000050,0.513897,0.897385,0.800000,715.0,URBANO,0.000000,0.000000,0.957475,...,0.000000,0.000000,0.000000,0.000000,0.349837,1.875474,0.000000,0.0,0.0,NaN
28088,zz2USdjR81hiUpG4IRTPwQ,410310705000015,0.528199,0.956389,0.800000,458.0,RURAL,1.007803,0.068867,0.108461,...,0.000000,0.000000,0.000000,0.000000,1.224538,0.000000,2.036786,0.0,0.0,NaN
28089,zz4P4Hi_eKM9OtoAL0ByNw,352610005000025,0.499226,0.902690,0.783333,427.0,RURAL,0.000000,0.000000,0.848774,...,0.362669,0.000000,0.000000,0.000000,0.196430,0.000000,0.000000,0.0,0.0,NaN
28090,zzZ7kabHUev787MQtij-Tg,352030105000050,0.696957,0.981552,0.816667,724.0,URBANO,0.357998,1.288575,0.000000,...,0.000000,1.532890,0.000000,0.267502,0.000000,0.000000,0.000000,0.0,0.0,NaN


In [39]:

# Step 3: Generate X
BAND_ORDER = ['BLUE', 'GREEN', 'RED', 'SWIR1', 'SWIR2', 'TEMP1', 'NIR', 'NIGHTLIGHTS', 'RGB']
column_names = []
for band in BAND_ORDER:
    column_names.extend(["v{}_{}".format(x, band) for x in range(4096)])

#X = df2[["v{}_x".format(x) for x in range(4096)] + ["v{}_y".format(x) for x in range(4096)] + column_names].copy()
#SV+SM+SAT
X = df2[["v{:d}".format(x) for x in range(4096)]+["v{:d}_0".format(x) for x in range(4096)]+["v{:d}_90".format(x) for x in range(4096)]+["v{:d}_180".format(x) for x in range(4096)]+["v{:d}_270".format(x) for x in range(4096)]+ column_names].copy()
#SV+SAT
#X = df2[["v{:d}_0".format(x) for x in range(4096)]+["v{:d}_90".format(x) for x in range(4096)]+["v{:d}_180".format(x) for x in range(4096)]+["v{:d}_270".format(x) for x in range(4096)]+ column_names].copy()
#SV
#X = df2[["v{:d}_0".format(x) for x in range(4096)]+["v{:d}_90".format(x) for x in range(4096)]+["v{:d}_180".format(x) for x in range(4096)]+["v{:d}_270".format(x) for x in range(4096)]].copy()
#SAT - RGB
#X = df2[["v{:d}_RGB".format(x) for x in range(4096)]].copy()
#SAT
#X = df2[column_names].copy()
#SM
#X = df2[["v{:d}".format(x) for x in range(4096)]].copy() 

X

,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v4086_RGB,v4087_RGB,v4088_RGB,v4089_RGB,v4090_RGB,v4091_RGB,v4092_RGB,v4093_RGB,v4094_RGB,v4095_RGB
0,1.075999,4.693594,1.523445,2.952631,2.659574,0.196765,0.458955,0.0,0.000294,0.000000,...,0.000000,0.000000,1.490444,0.000000,0.000000,1.729472,0.711890,0.0,0.000000,0.0
1,0.000000,0.912145,0.000000,2.219975,2.741114,0.000000,0.547258,0.0,0.000000,0.000000,...,0.439743,0.000000,2.003367,0.473483,0.000000,2.055765,0.505277,0.0,0.000000,0.0
2,0.000000,2.751529,0.000000,2.168793,2.253815,0.000000,0.000000,0.0,0.391170,0.000000,...,0.960934,0.000000,0.440303,0.496778,0.000000,1.079047,1.446692,0.0,0.000000,0.0
3,0.000000,0.275511,0.000000,0.000000,2.844376,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,1.604694,2.782040,0.000000,0.000000,3.410031,0.000000,0.0,0.000000,0.0
4,0.000000,2.421854,0.000000,2.738095,4.000751,0.000000,0.000000,0.0,0.000000,0.000000,...,1.211466,0.000000,2.155622,0.000000,0.000000,1.576869,0.888900,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28087,0.000000,2.543763,0.000000,0.339314,3.913337,0.000000,1.456961,0.0,0.000000,0.000000,...,0.429661,0.000000,0.818567,0.059769,0.000000,1.706120,1.378714,0.0,0.167554,0.0
28088,0.000000,1.991800,0.707980,0.000000,1.553528,0.000000,1.204969,0.0,0.000000,1.977652,...,0.846822,0.000000,0.994390,1.217486,0.273068,0.973584,1.487119,0.0,0.231716,0.0
28089,0.000000,1.260235,0.000000,0.569757,0.676200,0.000000,0.809055,0.0,0.000000,0.000000,...,0.554771,0.000000,1.097150,0.000000,0.000000,1.301685,0.000000,0.0,0.000000,0.0
28090,0.000000,0.000000,0.000000,0.555569,2.153547,0.556979,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,1.542355,0.000000,0.000000,0.000000,0.270714,0.0,0.000000,0.0


In [40]:
X = X.fillna(0)

In [41]:
# Step 4: Generate y

y = df2[indicator]
y

0        0.660533
1        0.735173
2        0.499030
3        0.525143
4        0.679411
           ...   
28087    0.513897
28088    0.528199
28089    0.499226
28090    0.696957
28091    0.576476
Name: income, Length: 27991, dtype: float64

In [42]:
y = y.fillna(0)

In [43]:

# Step 5: Predict y_pred
y_pred = loaded_model.predict(X)


In [44]:

# Step 6: Calculate metrics
r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))


In [45]:
r2

0.7521999404724181

In [46]:
mae

0.038572614272538344

In [47]:
rmse

0.050727871333396395

In [22]:
# Calculate the differences between y and y_pred
y_diff = y - y_pred

In [23]:

# Create a new DataFrame to store the differences and the "setor" column
diff_df = pd.DataFrame({'setor': df2['setor'], 'y_diff': y_diff})


In [24]:



# Group by "setor" and calculate the average of y_diff
avg_diff_by_setor = diff_df.groupby('setor')['y_diff'].mean()

# If you want the result as a DataFrame
avg_diff_by_setor_df = avg_diff_by_setor.reset_index()
avg_diff_by_setor_df


,setor,y_diff
0,350270505000002,-0.034236
1,350270505000003,0.065497
2,350270505000004,0.101263
3,350270505000006,0.075858
4,350270505000009,0.060332
...,...,...
542,412863305000008,-0.001428
543,412863305000009,-0.045642
544,412863305000010,-0.040272
545,412863305000011,-0.000131


In [25]:
avg_diff_by_setor_df.to_csv('diff_agg.csv' ,mode='a', index=None)